<a href="https://colab.research.google.com/github/rezatan/RockPaperScissors/blob/main/RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile,os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
#Download dataset
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
#Extract downloaded dataset to base directory
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
base_dir = '/tmp/rockpaperscissors/'

In [ ]:
#Split file with ratio 6:4 for train and validation data
!pip install split-folders
import split_folders
split_folders.ratio('/tmp/rockpaperscissors/rps-cv-images', output='/tmp/rockpaperscissors/hasil_split', seed=1337, ratio= (.6, .4))

Copying files: 2188 files [00:00, 3584.86 files/s]


In [ ]:
hasil_split_dir = os.path.join(base_dir,'hasil_split')
train_dir = os.path.join(hasil_split_dir,'train')
val_dir = os.path.join(hasil_split_dir,'val')
train_rock_dir = os.path.join(train_dir, 'rock')
train_paper_dir = os.path.join(train_dir, 'paper')
train_scissors_dir = os.path.join(train_dir, 'scissors')
val_rock_dir = os.path.join(val_dir, 'rock')
val_paper_dir = os.path.join(val_dir, 'paper')
val_scissors_dir = os.path.join(val_dir, 'scissors')

In [ ]:
#Declare train and test image data generator variable
train_datagen = ImageDataGenerator(rescale=1./255,
                              rotation_range = 20,
                              shear_range=0.2,
                              fill_mode='nearest',
                              horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255,
                              rotation_range = 20,
                              shear_range=0.2,
                              fill_mode='nearest',
                              horizontal_flip=True)

In [ ]:
#train and valid data in directory with image data generator and save it to new variable
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    )

val_generator = test_datagen.flow_from_directory(val_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='categorical',
                                                )

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [ ]:
#Make model layer
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax'),
])

In [ ]:
#Make model compiler
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.SGD(learning_rate=1e-4, momentum=0.99),
              metrics=['accuracy'])

In [ ]:
my_callbacks = [
                tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
  print(train_generator.class_indices)

{'paper': 0, 'rock': 1, 'scissors': 2}


In [ ]:
#Run Model to train AI
model.fit(
      train_generator,
      steps_per_epoch=28,
      epochs=29,
      validation_data=val_generator,
      validation_steps=28,
      callbacks = my_callbacks,
      verbose=2)

Epoch 1/29
28/28 - 61s - loss: 1.1015 - accuracy: 0.3471 - val_loss: 1.0982 - val_accuracy: 0.3345
Epoch 2/29
28/28 - 60s - loss: 1.0962 - accuracy: 0.3426 - val_loss: 1.0952 - val_accuracy: 0.3950
Epoch 3/29
28/28 - 62s - loss: 1.0955 - accuracy: 0.3571 - val_loss: 1.0907 - val_accuracy: 0.4247
Epoch 4/29
28/28 - 62s - loss: 1.0891 - accuracy: 0.4141 - val_loss: 1.0853 - val_accuracy: 0.5068
Epoch 5/29
28/28 - 61s - loss: 1.0861 - accuracy: 0.4141 - val_loss: 1.0787 - val_accuracy: 0.5297
Epoch 6/29
28/28 - 61s - loss: 1.0807 - accuracy: 0.4051 - val_loss: 1.0732 - val_accuracy: 0.4041
Epoch 7/29
28/28 - 61s - loss: 1.0715 - accuracy: 0.4319 - val_loss: 1.0653 - val_accuracy: 0.4817
Epoch 8/29
28/28 - 61s - loss: 1.0605 - accuracy: 0.4933 - val_loss: 1.0495 - val_accuracy: 0.6256
Epoch 9/29
28/28 - 62s - loss: 1.0507 - accuracy: 0.5614 - val_loss: 1.0321 - val_accuracy: 0.7466
Epoch 10/29
28/28 - 61s - loss: 1.0357 - accuracy: 0.5960 - val_loss: 1.0077 - val_accuracy: 0.6724
Epoch 11/

In [ ]:
#Detect rock paper scissors from pc uploaded image
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  
  print(fn)

  if (classes[0][0]==1): print('Paper')
  elif (classes[0][1]==1): print('Rock')
  elif (classes[0][2]==1): print('Scissors')